# Initialization

In [1]:
# add custom functions to path
import sys
sys.path.append("../src")

%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from ipywidgets import interact, fixed
import matplotlib.pyplot as plt
import seaborn as sns
import pystan
import pickle
from scipy.stats import gamma, norm, halfnorm
from stan import fit_model, plot_weibull_subject_fit
from stan import plot_posterior, plot_hierarchical_wald_fit

# load in behavioral data and clean
data = pd.read_csv('../data/derivatives/behavior/group_data.tsv', sep='\t', 
                   na_values='n/a')
exclusions = ['no_response', 'error', 'post_error', 'fast_rt']
data = data[data[exclusions].sum(axis=1) == 0]

subjects = sorted(list(data.participant_id.unique()))
print(subjects)

sns.set(style='whitegrid', font_scale=2)

['sub-hc001', 'sub-hc002', 'sub-hc003', 'sub-hc004', 'sub-hc005', 'sub-hc006', 'sub-hc007', 'sub-hc008', 'sub-hc009', 'sub-hc010', 'sub-hc011', 'sub-hc012', 'sub-hc013', 'sub-hc014', 'sub-hc015', 'sub-hc016', 'sub-hc017', 'sub-hc018', 'sub-hc019', 'sub-hc020', 'sub-hc021', 'sub-hc022', 'sub-hc023', 'sub-hc024', 'sub-hc025', 'sub-hc026', 'sub-hc027', 'sub-hc028', 'sub-hc029', 'sub-hc030', 'sub-hc031', 'sub-hc032', 'sub-hc033', 'sub-hc034', 'sub-hc035', 'sub-hc036', 'sub-hc037', 'sub-hc038', 'sub-hc041', 'sub-hc042', 'sub-hc044', 'sub-hc045', 'sub-hc047', 'sub-pp001', 'sub-pp002', 'sub-pp003', 'sub-pp004', 'sub-pp005', 'sub-pp006', 'sub-pp007', 'sub-pp008', 'sub-pp009', 'sub-pp010', 'sub-pp011', 'sub-pp012', 'sub-pp013', 'sub-pp014', 'sub-pp015', 'sub-pp016']


# Wald Model 

In [64]:
def wald(x, alpha, gamma, theta):
    first = alpha / np.sqrt(2 * np.pi * np.power(x - theta, 3))
    second = np.exp(-(np.power(alpha - gamma * (x - theta), 2)) / (2 * (x - theta)))
    return first * second

## Wald Intuition

In [17]:
def plot_wald(db, dr, ndt):
    x = np.arange(ndt, 1.75, .001, dtype=np.float64)
    plt.plot(x, wald(x, db, dr, ndt))
    plt.xlim((0, 1.75))
    plt.show();
    
interact(plot_wald, db=(0, 5, .01), dr=(0, 8, .01), ndt=(0, 2, .01));

interactive(children=(FloatSlider(value=2.0, description=u'db', max=5.0, step=0.01), FloatSlider(value=4.0, description=u'dr', max=8.0, step=0.01), FloatSlider(value=1.0, description=u'ndt', max=2.0, step=0.01), Output()), _dom_classes=('widget-interact',))

## Prior Intution & Determination

In [65]:
model = pystan.StanModel(file='../models/wald/single_subject_wald.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_517e849899abc658501bb29e8bc61bda NOW.


### Fit Subject MAPS

In [66]:
maps = []
for subject in subjects:
    print(subject)
    sub_data = data[data.participant_id == subject]
    rt_c = sub_data[sub_data.trial_type == 'congruent'].response_time
    rt_i = sub_data[sub_data.trial_type == 'incongruent'].response_time
    data_in = {'Ni': len(rt_i), 'Nc': len(rt_c), 'rt_c': rt_c, 'rt_i': rt_i}
    
    op = model.optimizing(data=data_in, seed=1000)
    maps.append(op)
print('Done!')

sub-hc001
sub-hc002
sub-hc003
sub-hc004
sub-hc005
sub-hc006
sub-hc007
sub-hc008
sub-hc009
sub-hc010
sub-hc011
sub-hc012
sub-hc013
sub-hc014
sub-hc015
sub-hc016
sub-hc017
sub-hc018
sub-hc019
sub-hc020
sub-hc021
sub-hc022
sub-hc023
sub-hc024
sub-hc025
sub-hc026
sub-hc027
sub-hc028
sub-hc029
sub-hc030
sub-hc031
sub-hc032
sub-hc033
sub-hc034
sub-hc035
sub-hc036
sub-hc037
sub-hc038
sub-hc041
sub-hc042
sub-hc044
sub-hc045
sub-hc047
sub-pp001
sub-pp002
sub-pp003
sub-pp004
sub-pp005
sub-pp006
sub-pp007
sub-pp008
sub-pp009
sub-pp010
sub-pp011
sub-pp012
sub-pp013
sub-pp014
sub-pp015
sub-pp016
Done!


In [20]:
def plot_subject_fit(subject):
    f, ax = plt.subplots(1, 1, figsize=(16, 8))
    colors = ['#e41a1c', '#377eb8']
    conditions = ['incongruent', 'congruent']
    sub_ix = subjects.index(subject)
    mapp = maps[sub_ix]
    sub_data = data[data.participant_id == subject]
    
    for i, c in enumerate(conditions):
        rt = sub_data[sub_data.trial_type == c].response_time
        x = np.arange(mapp['ndt_%s' % c[0]], 1.75, .01)
        sns.distplot(rt, color=colors[i], ax=ax, kde=False, 
                     norm_hist=True)
        print('%s = DB: %.2f, DR: %.2f, NDT: %.2f' % (c, mapp['db_%s' % c[0]],
                                                      mapp['dr_%s' % c[0]],
                                                      mapp['ndt_%s' % c[0]]))
        plt.plot(x, wald(x, mapp['db_%s' % c[0]], 
                            mapp['dr_%s' % c[0]], 
                            mapp['ndt_%s' % c[0]]), color=colors[i])
    
    plt.legend(conditions)
    plt.xlim((0, 1.75))
    plt.ylim((0, 4))
    plt.show()
    
interact(plot_subject_fit, subject=subjects);

interactive(children=(Dropdown(description=u'subject', options=('sub-hc001', 'sub-hc002', 'sub-hc003', 'sub-hc004', 'sub-hc005', 'sub-hc006', 'sub-hc007', 'sub-hc008', 'sub-hc009', 'sub-hc010', 'sub-hc011', 'sub-hc012', 'sub-hc013', 'sub-hc014', 'sub-hc015', 'sub-hc016', 'sub-hc017', 'sub-hc018', 'sub-hc019', 'sub-hc020', 'sub-hc021', 'sub-hc022', 'sub-hc023', 'sub-hc024', 'sub-hc025', 'sub-hc026', 'sub-hc027', 'sub-hc028', 'sub-hc029', 'sub-hc030', 'sub-hc031', 'sub-hc032', 'sub-hc033', 'sub-hc034', 'sub-hc035', 'sub-hc036', 'sub-hc037', 'sub-hc038', 'sub-hc041', 'sub-hc042', 'sub-hc044', 'sub-hc045', 'sub-hc047', 'sub-pp001', 'sub-pp002', 'sub-pp003', 'sub-pp004', 'sub-pp005', 'sub-pp006', 'sub-pp007', 'sub-pp008', 'sub-pp009', 'sub-pp010', 'sub-pp011', 'sub-pp012', 'sub-pp013', 'sub-pp014', 'sub-pp015', 'sub-pp016'), value='sub-hc001'), Output()), _dom_classes=('widget-interact',))

### Visualize Priors

In [67]:
def calc_rate(mode, sd):
    rate = (mode + np.sqrt(mode**2 + 4 * sd**2)) / (2 * sd**2)
    return rate

def calc_shape(mode, rate):
    shape = 1 + mode * rate
    return shape

def calc_mode(shape, rate):
    return (shape - 1) / rate

def plot_priors(param, group_theta_hyp_mode, group_theta_hyp_sd, 
                group_k_hyp_mode, group_k_hyp_sd):
    
    f, axs = plt.subplots(3, 1, figsize=(10, 8))
    colors = ['#e41a1c', '#377eb8']
   
    param_i_maps = [m['%s_i' % param] for m in maps]
    param_c_maps = [m['%s_c' % param] for m in maps]
    sns.distplot(param_c_maps, ax=axs[0], color=colors[1], kde=False, 
                 bins=10, norm_hist=True)
    sns.distplot(param_i_maps, ax=axs[0], color=colors[0], kde=False, 
                 bins=10, norm_hist=True)
    
    x = np.arange(0, 20, .01)
        
    group_k_hyp_theta = calc_rate(group_k_hyp_mode, group_k_hyp_sd)
    group_k_hyp_k = calc_shape(group_k_hyp_mode, group_k_hyp_theta)
    group_k_hyp_prior = gamma(group_k_hyp_k, scale=1/group_k_hyp_theta)
    axs[1].plot(x, group_k_hyp_prior.pdf(x))
    axs[1].axvline(group_k_hyp_mode)
    axs[1].set_title('Group K Hyperprior: k=%.2f, theta=%.2f' % (group_k_hyp_k,
                                                                 group_k_hyp_theta))
    
    group_theta_hyp_theta = calc_rate(group_theta_hyp_mode, group_theta_hyp_sd)
    group_theta_hyp_k = calc_shape(group_theta_hyp_mode, group_theta_hyp_theta)
    group_theta_hyp_prior = gamma(group_theta_hyp_k, scale=1/group_theta_hyp_theta)
    axs[2].plot(x, group_theta_hyp_prior.pdf(x))
    axs[2].axvline(group_theta_hyp_mode)
    axs[2].set_title('Group Theta Hyperprior: k=%.2f, theta=%.2f' % (group_theta_hyp_k,
                                                                 group_theta_hyp_theta))
    
    if param == 'ndt':
        x = np.arange(0, 1.75, .01, dtype=np.float64)
    else:
        x = np.arange(0, 15, .01, dtype=np.float64)
    
    group_mode = calc_mode(group_k_hyp_mode, group_theta_hyp_mode)
    group_prior = gamma(group_k_hyp_mode, scale=1/group_theta_hyp_mode)
    axs[0].plot(x, group_prior.pdf(x))
    axs[0].axvline(group_mode)
    
    plt.tight_layout()
    plt.show()

interact(plot_priors, param=['db', 'dr', 'ndt'], 
         group_theta_hyp_mode=(0, 20, .1), group_theta_hyp_sd=(0, 20, .1), 
         group_k_hyp_mode=(0, 20, .1), group_k_hyp_sd=(0, 20, .1));

interactive(children=(Dropdown(description=u'param', options=('db', 'dr', 'ndt'), value='db'), FloatSlider(value=10.0, description=u'group_theta_hyp_mode', max=20.0), FloatSlider(value=10.0, description=u'group_theta_hyp_sd', max=20.0), FloatSlider(value=10.0, description=u'group_k_hyp_mode', max=20.0), FloatSlider(value=10.0, description=u'group_k_hyp_sd', max=20.0), Output()), _dom_classes=('widget-interact',))

## Hierarchical Model Fitting

In [3]:
model = 'wald_hierarchical'

### Prepare Data

In [3]:
min_rts = np.array(data.groupby(['participant_id', 'trial_type']).response_time.min())
min_rt_i = min_rts[1::2]
min_rt_c = min_rts[::2]
ns = len(data.participant_id.unique())
data_i = data[data.trial_type == 'incongruent']
data_c = data[data.trial_type == 'congruent']
ll_i = data_i.participant_id.astype('category').cat.codes + 1
ll_c = data_c.participant_id.astype('category').cat.codes + 1
rt_i = data_i.response_time
rt_c = data_c.response_time
data_in = {'Ns': ns, 'll_i': ll_i, 'Ni': len(rt_i), 'rt_i': rt_i,
           'll_c': ll_c, 'Nc': len(rt_c), 'rt_c': rt_c,
           'min_rt_i': min_rt_i, 'min_rt_c': min_rt_c}

### Fit Model

In [4]:
n_iter = 1000
init_dict = {'dr_mode_congruent': 4, 'dr_mode_incongruent': 4,
             'dr_sd_congruent': 2.5, 'dr_sd_incongruent': 2.5,
             'db_mode_congruent': 1.5, 'db_sd_congruent': 3,
             'db_mode_incongruent': 1.5, 'db_sd_incongruent': 1.5,
             'ndt_mode_congruent': .3, 'ndt_sd_congruent': .2,
             'ndt_mode_incongruent': .3, 'ndt_sd_incongruent': .2,
             'dr_congruent': [4] * ns, 'dr_incongruent': [4] * ns,
             'db_congruent': [1.5] * ns, 'db_incongruent': [1.5] * ns,
             'ndt_congruent': [.3] * ns, 'ndt_incongruent': [.3] * ns}
model_fit = fit_model(model, 'wald', data_in, n_iter=n_iter, 
                      seed=7, init=init_dict, n_chains=3)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL wald_hierarchical_f873946d9486dc5910fb2422f444ae11 NOW.


Starting Model Fit...
Compiling Model...
Compiling took 0 min. 39 sec.
Computing MAP Estimates...
Finding MAP estimates took 0 min. 8 sec.
Sampling from Posterior...
Drawing 1000 Posterior Samples took 92 min. 48 sec.
Extracting Samples...
Extracting samples took 0 min. 0 sec.
Extracting Fit Summary...
Extracting fit summary took 0 min. 1 sec.
Pickling Model Fit...
Pickling model fit took 0 min. 0 sec.
Total Time: took 93 min. 37 sec.
Finished


### Visualize Results

In [4]:
model_fit = pickle.load(open('../models/wald/%s.pkl' % model, 'r'))

In [5]:
interact(plot_posterior, param=model_fit['map'].keys(), model_fit=fixed(model_fit),
         subject=subjects, subjects=fixed(subjects));

interactive(children=(Dropdown(description=u'param', options=(u'dr_group_k_congruent', u'dr_group_theta_congruent', u'dr_group_k_incongruent', u'dr_group_theta_incongruent', u'db_group_k_congruent', u'db_group_theta_congruent', u'db_group_k_incongruent', u'db_group_theta_incongruent', u'ndt_group_k_congruent', u'ndt_group_theta_congruent', u'ndt_group_k_incongruent', u'ndt_group_theta_incongruent', u'ndt_inc', u'ndt_con', u'dr_congruent', u'dr_incongruent', u'db_congruent', u'db_incongruent', u'ndt_incongruent', u'ndt_congruent', u'beta_ndt', u'beta_db', u'beta_dr', u'dr_group_mode_incongruent', u'dr_group_mode_congruent', u'db_group_mode_incongruent', u'db_group_mode_congruent', u'ndt_group_mode_incongruent', u'ndt_group_mode_congruent', u'group_beta_ndt', u'group_beta_db', u'group_beta_dr'), value=u'dr_group_k_congruent'), Dropdown(description=u'subject', options=('sub-hc001', 'sub-hc002', 'sub-hc003', 'sub-hc004', 'sub-hc005', 'sub-hc006', 'sub-hc007', 'sub-hc008', 'sub-hc009', 'sub-hc010', 'sub-hc011', 'sub-hc012', 'sub-hc013', 'sub-hc014', 'sub-hc015', 'sub-hc016', 'sub-hc017', 'sub-hc018', 'sub-hc019', 'sub-hc020', 'sub-hc021', 'sub-hc022', 'sub-hc023', 'sub-hc024', 'sub-hc025', 'sub-hc026', 'sub-hc027', 'sub-hc028', 'sub-hc029', 'sub-hc030', 'sub-hc031', 'sub-hc032', 'sub-hc033', 'sub-hc034', 'sub-hc035', 'sub-hc036', 'sub-hc037', 'sub-hc038', 'sub-hc041', 'sub-hc042', 'sub-hc044', 'sub-hc045', 'sub-hc047', 'sub-pp001', 'sub-pp002', 'sub-pp003', 'sub-pp004', 'sub-pp005', 'sub-pp006', 'sub-pp007', 'sub-pp008', 'sub-pp009', 'sub-pp010', 'sub-pp011', 'sub-pp012', 'sub-pp013', 'sub-pp014', 'sub-pp015', 'sub-pp016'), value='sub-hc001'), Output()), _dom_classes=('widget-interact',))

In [73]:
interact(plot_hierarchical_wald_fit, model_fit=fixed(model_fit), 
         behavior=fixed(data), subject=subjects, subjects=fixed(subjects));

interactive(children=(Dropdown(description=u'subject', options=('sub-hc001', 'sub-hc002', 'sub-hc003', 'sub-hc004', 'sub-hc005', 'sub-hc006', 'sub-hc007', 'sub-hc008', 'sub-hc009', 'sub-hc010', 'sub-hc011', 'sub-hc012', 'sub-hc013', 'sub-hc014', 'sub-hc015', 'sub-hc016', 'sub-hc017', 'sub-hc018', 'sub-hc019', 'sub-hc020', 'sub-hc021', 'sub-hc022', 'sub-hc023', 'sub-hc024', 'sub-hc025', 'sub-hc026', 'sub-hc027', 'sub-hc028', 'sub-hc029', 'sub-hc030', 'sub-hc031', 'sub-hc032', 'sub-hc033', 'sub-hc034', 'sub-hc035', 'sub-hc036', 'sub-hc037', 'sub-hc038', 'sub-hc041', 'sub-hc042', 'sub-hc044', 'sub-hc045', 'sub-hc047', 'sub-pp001', 'sub-pp002', 'sub-pp003', 'sub-pp004', 'sub-pp005', 'sub-pp006', 'sub-pp007', 'sub-pp008', 'sub-pp009', 'sub-pp010', 'sub-pp011', 'sub-pp012', 'sub-pp013', 'sub-pp014', 'sub-pp015', 'sub-pp016'), value='sub-hc001'), Output()), _dom_classes=('widget-interact',))

# Weibull Model

Main parameters with interpretations from Rouder 2005:
- shape: Change in structure of central processing. Such as different overall processing strategy.
- scale: Differences in speed of central processing given similar structure.
- shift: Differences in peripheral processing (motor, visual, etc.).


In [2]:
def weibull(x, alpha, sigma, shift):
    p1 = (alpha / sigma)
    p2 = np.power((x - shift) / sigma, alpha - 1) 
    p3 = np.exp(-np.power((x - shift) / sigma, alpha))
    return p1 * p2 * p3

## Weibull Intuition

In [10]:
def plot_weibull(shape, scale, shift):
    x = np.arange(shift, 1.75, .001, dtype=np.float64)
    plt.plot(x, weibull(x, shape, scale, shift))
    plt.xlim((0, 1.75))
    plt.show();
    
interact(plot_weibull, shape=(1, 5, .01), scale=(0, 2, .01), shift=(0, 2, .01));

interactive(children=(FloatSlider(value=3.0, description=u'shape', max=5.0, min=1.0, step=0.01), FloatSlider(value=1.0, description=u'scale', max=2.0, step=0.01), FloatSlider(value=1.0, description=u'shift', max=2.0, step=0.01), Output()), _dom_classes=('widget-interact',))

## Prior Intuition & Determination

### Build distribution of individual subject MAPs

First, we compile the inidividual subject stan model.

In [8]:
model = pystan.StanModel(file='../models/weibull/single_subject_weibull.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_8b64186d7c72474f92436047d86f728a NOW.


Next we gather the map estimates for each subject's shift, scale, and shape parameters split by condition. 

In [9]:
maps = []
for subject in subjects:
    print(subject)
    sub_data = data[data.participant_id == subject]
    rt_c = sub_data[sub_data.trial_type == 'congruent'].response_time
    rt_i = sub_data[sub_data.trial_type == 'incongruent'].response_time
    data_in = {'Ni': len(rt_i), 'Nc': len(rt_c), 'rt_c': rt_c, 'rt_i': rt_i}
    
    op = model.optimizing(data=data_in, seed=8)
    maps.append(op)
print('Done!')

sub-hc001
sub-hc002
sub-hc003
sub-hc004
sub-hc005
sub-hc006
sub-hc007
sub-hc008
sub-hc009
sub-hc010
sub-hc011
sub-hc012
sub-hc013
sub-hc014
sub-hc015
sub-hc016
sub-hc017
sub-hc018
sub-hc019
sub-hc020
sub-hc021
sub-hc022
sub-hc023
sub-hc024
sub-hc025
sub-hc026
sub-hc027
sub-hc028
sub-hc029
sub-hc030
sub-hc031
sub-hc032
sub-hc033
sub-hc034
sub-hc035
sub-hc036
sub-hc037
sub-hc038
sub-hc041
sub-hc042
sub-hc044
sub-hc045
sub-hc047
sub-pp001
sub-pp002
sub-pp003
sub-pp004
sub-pp005
sub-pp006
sub-pp007
sub-pp008
sub-pp009
sub-pp010
sub-pp011
sub-pp012
sub-pp013
sub-pp014
sub-pp015
sub-pp016
Done!


Next, we sift through the map distribution fits to each subject's data as a sanity check of how well we can fit.

In [7]:
def plot_subject_fit(subject):
    f, ax = plt.subplots(1, 1, figsize=(16, 8))
    colors = ['#e41a1c', '#377eb8']
    conditions = ['incongruent', 'congruent']
    sub_ix = subjects.index(subject)
    mapp = maps[sub_ix]
    sub_data = data[data.participant_id == subject]
    
    for i, c in enumerate(conditions):
        rt = sub_data[sub_data.trial_type == c].response_time
        x = np.arange(mapp['shift_%s' % c[0]], 1.75, .01)
        sns.distplot(rt, color=colors[i], ax=ax, kde=False, 
                     norm_hist=True)
        plt.plot(x, weibull(x, mapp['shape_%s' % c[0]], 
                            mapp['scale_%s' % c[0]], 
                            mapp['shift_%s' % c[0]]), color=colors[i])
    
    plt.legend(conditions)
    plt.xlim((0, 1.75))
    plt.ylim((0, 4))
    plt.show()
    
interact(plot_subject_fit, subject=subjects);

interactive(children=(Dropdown(description=u'subject', options=('sub-hc001', 'sub-hc002', 'sub-hc003', 'sub-hc004', 'sub-hc005', 'sub-hc006', 'sub-hc007', 'sub-hc008', 'sub-hc009', 'sub-hc010', 'sub-hc011', 'sub-hc012', 'sub-hc013', 'sub-hc014', 'sub-hc015', 'sub-hc016', 'sub-hc017', 'sub-hc018', 'sub-hc019', 'sub-hc020', 'sub-hc021', 'sub-hc022', 'sub-hc023', 'sub-hc024', 'sub-hc025', 'sub-hc026', 'sub-hc027', 'sub-hc028', 'sub-hc029', 'sub-hc030', 'sub-hc031', 'sub-hc032', 'sub-hc033', 'sub-hc034', 'sub-hc035', 'sub-hc036', 'sub-hc037', 'sub-hc038', 'sub-hc041', 'sub-hc042', 'sub-hc044', 'sub-hc045', 'sub-hc047', 'sub-pp001', 'sub-pp002', 'sub-pp003', 'sub-pp004', 'sub-pp005', 'sub-pp006', 'sub-pp007', 'sub-pp008', 'sub-pp009', 'sub-pp010', 'sub-pp011', 'sub-pp012', 'sub-pp013', 'sub-pp014', 'sub-pp015', 'sub-pp016'), value='sub-hc001'), Output()), _dom_classes=('widget-interact',))

### Visualizing Prior Distribution

For the hiearchical model, we are not going to model each condition with a separate distribution. Instead, we will do a regression approach.

First we, look at the priors for the scale parameter.

In [57]:
def calc_rate(mode, sd):
    rate = (mode + np.sqrt(mode**2 + 4 * sd**2)) / (2 * sd**2)
    return rate

def calc_shape(mode, rate):
    shape = 1 + mode * rate
    return shape

def calc_mode(shape, rate):
    return (shape - 1) / rate

def plot_priors(param, group_theta_hyp_mode, group_theta_hyp_sd, 
                group_k_hyp_mode, group_k_hyp_sd):
    
    f, axs = plt.subplots(3, 1, figsize=(10, 8))
    colors = ['#e41a1c', '#377eb8']
   
        
    param_i_maps = [m['%s_i' % param] for m in maps]
    param_c_maps = [m['%s_c' % param] for m in maps]
    sns.distplot(param_c_maps, ax=axs[0], color=colors[1], kde=False, 
                 bins=10, norm_hist=True)
    sns.distplot(param_i_maps, ax=axs[0], color=colors[0], kde=False, 
                 bins=10, norm_hist=True)
    
    x = np.arange(0, 20, .01)
        
    group_k_hyp_theta = calc_rate(group_k_hyp_mode, group_k_hyp_sd)
    group_k_hyp_k = calc_shape(group_k_hyp_mode, group_k_hyp_theta)
    group_k_hyp_prior = gamma(group_k_hyp_k, scale=1/group_k_hyp_theta)
    axs[1].plot(x, group_k_hyp_prior.pdf(x))
    axs[1].axvline(group_k_hyp_mode)
    axs[1].set_title('Group K Hyperprior: k=%.2f, theta=%.2f' % (group_k_hyp_k,
                                                                 group_k_hyp_theta))
    
    group_theta_hyp_theta = calc_rate(group_theta_hyp_mode, group_theta_hyp_sd)
    group_theta_hyp_k = calc_shape(group_theta_hyp_mode, group_theta_hyp_theta)
    group_theta_hyp_prior = gamma(group_theta_hyp_k, scale=1/group_theta_hyp_theta)
    axs[2].plot(x, group_theta_hyp_prior.pdf(x))
    axs[2].axvline(group_theta_hyp_mode)
    axs[2].set_title('Group Theta Hyperprior: k=%.2f, theta=%.2f' % (group_theta_hyp_k,
                                                                 group_theta_hyp_theta))
    
    if param == 'shift':
        x = np.arange(0, 1.75, .01, dtype=np.float64)
    elif param == 'shape':
        x = np.arange(0, 7, .01, dtype=np.float64)
    else:
        x = np.arange(0, 1, .01, dtype=np.float64)
    
    group_mode = calc_mode(group_k_hyp_mode, group_theta_hyp_mode)
    group_prior = gamma(group_k_hyp_mode, scale=1/group_theta_hyp_mode)
    axs[0].plot(x, group_prior.pdf(x))
    axs[0].axvline(group_mode)
    
    plt.tight_layout()
    plt.show()

interact(plot_priors, param=['shape', 'scale', 'shift'], 
         group_theta_hyp_mode=(0, 20, .1), group_theta_hyp_sd=(0, 20, .1), 
         group_k_hyp_mode=(0, 20, .1), group_k_hyp_sd=(0, 20, .1));

interactive(children=(Dropdown(description=u'param', options=('shape', 'scale', 'shift'), value='shape'), FloatSlider(value=10.0, description=u'group_theta_hyp_mode', max=20.0), FloatSlider(value=10.0, description=u'group_theta_hyp_sd', max=20.0), FloatSlider(value=10.0, description=u'group_k_hyp_mode', max=20.0), FloatSlider(value=10.0, description=u'group_k_hyp_sd', max=20.0), Output()), _dom_classes=('widget-interact',))

## Hierarchical Model Fitting 

In [6]:
model = 'weibull_hierarchical'

### Prepare Data

In [6]:
min_rts = np.array(data.groupby(['participant_id', 'trial_type']).response_time.min())
min_rt_i = min_rts[1::2]
min_rt_c = min_rts[::2]
ns = len(data.participant_id.unique())
data_i = data[data.trial_type == 'incongruent']
data_c = data[data.trial_type == 'congruent']
ll_i = data_i.participant_id.astype('category').cat.codes + 1
ll_c = data_c.participant_id.astype('category').cat.codes + 1
rt_i = data_i.response_time
rt_c = data_c.response_time
data_in = {'Ns': ns, 'll_i': ll_i, 'Ni': len(rt_i), 'rt_i': rt_i,
           'll_c': ll_c, 'Nc': len(rt_c), 'rt_c': rt_c,
           'min_rt_i': min_rt_i, 'min_rt_c': min_rt_c}

### Compile & Fit Model

In [7]:
n_iter = 1000
n_chains = 3
model_fit = fit_model(model, 'weibull', data_in, n_iter=n_iter, 
                      seed=7, init='random', n_chains=n_chains)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL weibull_hierarchical_5954055af205493781530fbe4b1191ec NOW.


Starting Model Fit...
Compiling Model...
Compiling took 0 min. 37 sec.
Computing MAP Estimates...
Finding MAP estimates took 0 min. 4 sec.
Sampling from Posterior...
Drawing 1000 Posterior Samples took 27 min. 39 sec.
Extracting Samples...
Extracting samples took 0 min. 0 sec.
Extracting Fit Summary...
Extracting fit summary took 0 min. 1 sec.
Pickling Model Fit...
Pickling model fit took 0 min. 0 sec.
Total Time: took 28 min. 23 sec.
Finished


### Plot Results

In [7]:
model_fit = pickle.load(open('../models/weibull/%s.pkl' % model, 'r'))

#### Plot the Posteriors

In [8]:
interact(plot_posterior, param=model_fit['map'].keys(), model_fit=fixed(model_fit),
         subject=subjects, subjects=fixed(subjects));

interactive(children=(Dropdown(description=u'param', options=(u'scale_group_k_congruent', u'scale_group_theta_congruent', u'scale_group_k_incongruent', u'scale_group_theta_incongruent', u'shape_group_k_congruent', u'shape_group_theta_congruent', u'shape_group_k_incongruent', u'shape_group_theta_incongruent', u'shift_group_k_congruent', u'shift_group_theta_congruent', u'shift_group_k_incongruent', u'shift_group_theta_incongruent', u'shift_inc', u'shift_con', u'scale_congruent', u'scale_incongruent', u'shape_congruent', u'shape_incongruent', u'shift_incongruent', u'shift_congruent', u'beta_shift', u'beta_shape', u'beta_scale', u'scale_group_mode_incongruent', u'scale_group_mode_congruent', u'shape_group_mode_incongruent', u'shape_group_mode_congruent', u'shift_group_mode_incongruent', u'shift_group_mode_congruent', u'group_beta_shape', u'group_beta_scale', u'group_beta_shift'), value=u'scale_group_k_congruent'), Dropdown(description=u'subject', options=('sub-hc001', 'sub-hc002', 'sub-hc003', 'sub-hc004', 'sub-hc005', 'sub-hc006', 'sub-hc007', 'sub-hc008', 'sub-hc009', 'sub-hc010', 'sub-hc011', 'sub-hc012', 'sub-hc013', 'sub-hc014', 'sub-hc015', 'sub-hc016', 'sub-hc017', 'sub-hc018', 'sub-hc019', 'sub-hc020', 'sub-hc021', 'sub-hc022', 'sub-hc023', 'sub-hc024', 'sub-hc025', 'sub-hc026', 'sub-hc027', 'sub-hc028', 'sub-hc029', 'sub-hc030', 'sub-hc031', 'sub-hc032', 'sub-hc033', 'sub-hc034', 'sub-hc035', 'sub-hc036', 'sub-hc037', 'sub-hc038', 'sub-hc041', 'sub-hc042', 'sub-hc044', 'sub-hc045', 'sub-hc047', 'sub-pp001', 'sub-pp002', 'sub-pp003', 'sub-pp004', 'sub-pp005', 'sub-pp006', 'sub-pp007', 'sub-pp008', 'sub-pp009', 'sub-pp010', 'sub-pp011', 'sub-pp012', 'sub-pp013', 'sub-pp014', 'sub-pp015', 'sub-pp016'), value='sub-hc001'), Output()), _dom_classes=('widget-interact',))

#### Plot the MAP Fits

In [16]:
interact(plot_weibull_subject_fit, model_name=fixed(model), model_fit=fixed(model_fit), 
         behavior=fixed(data), subject=subjects, subjects=fixed(subjects));

interactive(children=(Dropdown(description=u'subject', options=('sub-hc001', 'sub-hc002', 'sub-hc003', 'sub-hc004', 'sub-hc005', 'sub-hc006', 'sub-hc007', 'sub-hc008', 'sub-hc009', 'sub-hc010', 'sub-hc011', 'sub-hc012', 'sub-hc013', 'sub-hc014', 'sub-hc015', 'sub-hc016', 'sub-hc017', 'sub-hc018', 'sub-hc019', 'sub-hc020', 'sub-hc021', 'sub-hc022', 'sub-hc023', 'sub-hc024', 'sub-hc025', 'sub-hc026', 'sub-hc027', 'sub-hc028', 'sub-hc029', 'sub-hc030', 'sub-hc031', 'sub-hc032', 'sub-hc033', 'sub-hc034', 'sub-hc035', 'sub-hc036', 'sub-hc037', 'sub-hc038', 'sub-hc041', 'sub-hc042', 'sub-hc044', 'sub-hc045', 'sub-hc047', 'sub-pp001', 'sub-pp002', 'sub-pp003', 'sub-pp004', 'sub-pp005', 'sub-pp006', 'sub-pp007', 'sub-pp008', 'sub-pp009', 'sub-pp010', 'sub-pp011', 'sub-pp012', 'sub-pp013', 'sub-pp014', 'sub-pp015', 'sub-pp016'), value='sub-hc001'), Output()), _dom_classes=('widget-interact',))